In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
import pyspark.sql.functions as func

In [2]:
#Creating the Spark Session
spark = SparkSession.builder.appName("FisrtApp").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/19 17:25:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#DF schema
schema = StructType([\
    StructField("userID", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("friends", IntegerType(), True),
    ])

In [4]:
#Read CSV file
df_people = spark.read.format("csv")\
    .schema(schema)\
    .option("path", "/app/fakefriends.csv")\
    .load()

In [5]:
#Transformations
output = df_people.select(df_people.userID, df_people.name,\
                          df_people.age, df_people.friends)\
                          .where(df_people.age < 30).withColumn('insert_ts', func.current_timestamp())\
                          .orderBy(df_people.userID)

#Schema in DDL
#schema = "userID INT, name STRING, age INT, friends INT"

In [6]:
print(output.count())

112


In [7]:
#Creating a Temp View
output.createOrReplaceTempView("people")

In [ ]:
#Simple query
spark.sql('''
          SELECT *
          FROM people
          LIMIT 5
          ''').show()

+------+--------+---+-------+--------------------+
|userID|    name|age|friends|           insert_ts|
+------+--------+---+-------+--------------------+
|     1|Jean-Luc| 26|      2|2025-01-19 17:25:...|
|     9|    Hugh| 27|    181|2025-01-19 17:25:...|
|    16|  Weyoun| 22|    323|2025-01-19 17:25:...|
|    21|   Miles| 19|    268|2025-01-19 17:25:...|
|    24|  Julian| 25|      1|2025-01-19 17:25:...|
+------+--------+---+-------+--------------------+

